In [2]:
%matplotlib notebook

In [3]:
import numpy as np
import matplotlib.pyplot as plt

from ase import Atoms, io
from ase.build import molecule, make_supercell
from ase.quaternions import Quaternion
from muspinsim.spinsys import SpinSystem, MuonSpinSystem
from muspinsim.spinop import DensityOperator
from muspinsim.experiment import MuonExperiment
from muspinsim.constants import MU_GAMMA
from soprano.selection import AtomSelection
from soprano.properties.linkage import Molecules
from soprano.properties.transform import Rotate
from soprano.nmr.utils import _dip_constant, _dip_tensor
from soprano.nmr.tensor import NMRTensor
from soprano.calculate.powder import ZCW, SHREWD
from soprano.calculate.dipolar.randsums import _scalar_sum_distribution
from soprano.data.nmr import (_get_isotope_data, _get_nmr_data, _el_iso)
from soprano.utils import periodic_bridson, minimum_periodic, minimum_supcell, supcell_gridgen
from soprano.properties.nmr.dipolar import DipolarRSS, DipolarTensor, DipolarCoupling
from scipy.interpolate import interp1d

$$
\begin{align}
H_{jk} = &D_{jk}(\hat{z}\otimes\hat{z})-\frac{D_{jk}}{2}(1+\eta)(\hat{x}\otimes\hat{x})-\frac{D_{jk}}{2}(1-\eta)(\hat{y}\otimes\hat{y}) \\
= &D_{jk}(\hat{z}\otimes\hat{z})-\frac{D_{jk}}{2}(\mathbb{I}-\hat{z}\otimes\hat{z})+\frac{D_{jk}}{2}\eta(\hat{y}\otimes\hat{y}-\hat{x}\otimes\hat{x}) \\
= &\frac{3}{2}D_{jk}(1+\frac{\eta}{3})(\hat{z}\otimes\hat{z}) -\frac{D_{jk}}{2}(1+\eta)\mathbb{I} +D\eta\hat{y}\otimes\hat{y}
\end{align}
$$

$$
$$

Taking only the secular terms:

$$
\begin{align}
\mathbf{S}_j(\hat{z}\otimes\hat{z})\mathbf{S}_k \approx &\alpha_z^2S_j^{x}S_k^{x}+\beta_z^2S_j^{y}S_k^{y}+\gamma_z^2S_j^{z}S_k^{z} \\
= &\gamma_z^2 S_j^{z}S_k^{z} + \frac{\alpha_z^2}{4}(S_j^{+}+S_j^{-})(S_k^{+}+S_k^{-})-\frac{\beta_z^2}{4}(S_j^{+}-S_j^{-})(S_k^{+}-S_k^{-}) \\
\approx & \gamma_z^2 S_j^{z}S_k^{z}+ \frac{1}{4}(\alpha_z^2+\beta_z^2)(S_j^{+}S_k^{-}+S_j^{-}S_k^{+}) \\
= & \gamma_z^2 S_j^{z}S_k^{z}+ \frac{1}{2}(1-\gamma_z^2)(\mathbf{S}_j\mathbf{S}_k-S_j^{z}S_k^{z}) \\
\approx & (\frac{3}{2}\gamma_z^2-1)S_j^{z}S_k^{z}
\end{align} 
$$

The last line is possible because we know that $\mathbf{S}_j\mathbf{S}_k$ commutes with the operators we care about and so there's no need to retain it.

$$
B_{jk} = \frac{3}{2}D_{jk}\left(1+\frac{\eta}{3}\right)\left[\frac{3}{2}\gamma_{jk}^2-\frac{1}{2}\right]+D_{jk}\eta\left[\frac{3}{2}\beta_{jk}^2-\frac{1}{2}\right]
$$

The square:

$$
B_{jk}^2 = \frac{9}{4}D_{jk}^2\left(1+\frac{\eta}{3}\right)^2\left[\frac{9}{4}\gamma_{jk}^4-\frac{3}{2}\gamma_{jk}^2+\frac{1}{4}\right]+D_{jk}^2\eta^2\left[\frac{9}{4}\beta_{jk}^4-\frac{3}{2}\beta_{jk}^2+\frac{1}{4}\right]+3D_{jk}^2\left(1+\frac{\eta}{3}\right)\eta\left[\frac{9}{4}\gamma_{jk}^2\beta_{jk}^2-\frac{3}{4}\gamma_{jk}^2-\frac{3}{4}\beta_{jk}^2+\frac{1}{4}\right]
$$

Averaged over all directions this becomes:

$$
\left<B_{jk}^2\right> = \frac{9}{20}D_{jk}^2\left(1+\frac{\eta}{3}\right)^2+\frac{1}{5}D_{jk}^2\eta^2-\frac{3}{10}D_{jk}^2\left(1+\frac{\eta}{3}\right)\eta
$$

In [4]:
# The model system will be a octahedral hydrogen cluster

pos = np.array(np.meshgrid(*[[-1,0,1]]*3)).reshape((3,-1)).T
pos = pos[np.where(np.linalg.norm(pos, axis=1) == 1)]*0.5

a = Atoms('H'*len(pos), positions=pos, cell=np.eye(3)*10)
# rot = Rotate(quaternion=Quaternion.from_axis_angle([0,1,0.5], 1.3))
# a = rot(a)
assert(len(Molecules.get(a)) == 1)
io.write('clusterH.cif', a)

In [5]:
# Add disorder
s = 0.05
n = 3
disorder = np.random.normal(scale=s, size=(len(a),n,3))
a.set_array('disorder', disorder)

In [6]:
def get_average_diptens(p0, p1, g0, g1):
    # p0 and p1 are a list of positions
    p0 = np.array(p0)
    p1 = np.array(p1)
    
    r = p1[:,None,:]-p0[None,:,:]
    rnorm = np.linalg.norm(r, axis=-1)
    r /= rnorm[:,:,None]
    d = _dip_constant(rnorm*1e-10, g0, g1)

    D = d[:,:,None,None]*(3*r[:, :, None, :]*r[:, :, :, None]-np.eye(3)[None,None])
    D = np.average(D, axis=(0,1))
    return D

In [7]:
gH = _get_isotope_data('H', 'gamma')
D = get_average_diptens([[0,0,0],[0,0.1,0]], [[1,0,0]], gH, gH)
D = NMRTensor(D, order=NMRTensor.ORDER_HAEBERLEN)

In [14]:
class VanVleckDisordered(object):
    
    def __init__(self, atoms):
        self.atoms = atoms
        #self.Dc = DipolarCoupling.get(atoms)
        n = len(atoms)
        pairs = [(i,j) for i in range(n) for j in range(i+1,n)]
        gammas = _get_isotope_data(atoms.get_chemical_symbols(), 'gamma')
        positions = atoms.get_positions()
        disorder = atoms.get_array('disorder')
        posd = positions[:,None,:] + disorder
        self.D = {}
        self.Dc = {}
        for k in pairs:
            i, j = k
            D = get_average_diptens(posd[i], posd[j], gammas[i], gammas[j])*1e-3
            D = NMRTensor(D, order=NMRTensor.ORDER_HAEBERLEN)            
            self.D[k] = D
            # Now the important eigenvalues
            d = D.eigenvalues[2]
            eta = D.asymmetry
            z = D.eigenvectors[:,2]
            y = D.eigenvectors[:,1]
            self.Dc[k] = (d, z, eta, y)
        
        # Calculate all Bjks
        self.Bvals = [[0 if i == j else self.Bjk(i,j) for j in range(len(atoms))] for i in range(len(atoms))]
        self.Bvals = np.array(self.Bvals)
        
        self.I = _get_isotope_data(self.atoms.get_chemical_symbols(), 'I')
    
    # Here following the nomenclature from Van Vleck's paper    
    def Bjk(self, j, k, axis=[0,0,1]):
        axis = np.array(axis).astype(float)
        axis /= np.linalg.norm(axis)

        jk = tuple(sorted([j, k]))        
        Dc = self.Dc[jk]
        ct = np.dot(axis, Dc[1])
        cp = np.dot(axis, Dc[3])
        
        return Dc[0]*(1.5*(1+Dc[2]/3)*(3*ct**2-1)/2+Dc[2]*(3*cp**2-1)/2.0)

    def std2_single(self, axis=[0,0,1]):
        I = self.I
        B = [[0 if i == j else self.Bjk(i,j, axis) for j in range(len(I))] for i in range(len(I))]
        B = np.array(B)
        return np.sum(I[:,None]*(I[None,:]+1.0)*B**2/3.0)/len(I)
    
    def std2_powder(self):
        I = self.I
        n = len(I)
        B2 = np.zeros((n,n))
        for i in range(n):
            for j in range(n):
                if i == j:
                    continue
                Dc = self.Dc[tuple(sorted((i,j)))]
                d, eta = Dc[0], Dc[2]
                B2[i,j] = d**2*(9/4+3/4*eta**2)
                
        return np.sum(I[:,None]*(I[None,:]+1.0)*B2/15.0)/len(I)
    
    def dipfreqs(self, B=[0,0,1]):
        
        B = np.array(B)
        atoms = self.atoms
        ssys = SpinSystem(atoms.get_chemical_symbols())
        
        for i in range(len(atoms)):
            ssys.add_linear_term(i, B*ssys.gamma(i)*1e3)
            for j in range(i+1, len(ssys)):
                ssys.add_bilinear_term(i, j, self.D[(i,j)].data)
                
        H = ssys.hamiltonian
        Sxtot = sum([ssys.operator({i: 'x'}) for i in range(len(ssys))], 0*ssys.operator({}))
        evals, evecs = H.diag()
        Sxdiag = Sxtot.basis_change(evecs)

        freqs = (evals[:,None]-evals[None,:]).reshape((-1,))
        weights = np.abs(Sxdiag.matrix.T.reshape((-1,)))

        # Central frequency?
        larmor = np.linalg.norm(B)*ssys.gamma(0)*1e3
        # Which ones are close enough?
        peak_inds = np.where((abs(freqs-larmor) < 0.5*larmor)*(1-np.isclose(weights, 0)))
        
        return (freqs[peak_inds]-larmor), weights[peak_inds]

In [15]:
vvleck = VanVleckDisordered(a)
freqs, weights = vvleck.dipfreqs()
sigma = (vvleck.std2_single()**0.5)
sigmafreq = (np.sum(weights**2*freqs**2)/np.sum(weights**2))**0.5

print(sigma)
print(sigmafreq)

448.9132444963204
448.9802129801245


In [16]:
# Powder version?

pwd = ZCW('sphere')
orients, weights = pwd.get_orient_points(100)

sigmapwd = vvleck.std2_powder()**0.5
sigmaorient = np.average([vvleck.std2_single(p) for p in orients], weights=weights)**0.5


orientfreqs = np.array([])
orientweights = np.array([])

for o, w in zip(orients, weights):
   ofreqs, oweights = vvleck.dipfreqs(5.0*o)
   orientfreqs = np.concatenate([orientfreqs, ofreqs])
   orientweights = np.concatenate([orientweights, oweights*w])


sigmaorientfreq = (np.sum(orientweights**2*orientfreqs**2)/np.sum(orientweights**2))**0.5

print(sigmapwd)
print(sigmaorient)
print(sigmaorientfreq)

500.99114755088885
501.0207115070033
499.6272631756669


In [258]:
vvleck.Bjk(0,1)

509.62645715726273